# MIP model implementation details and notes

Implementation with the Python, solver independent, **PuLP** library. Key classes:
- LpProblem: MIP problem constructor.
- LpVariables: MIP variables.
- constraints and objective function are added with the overloaded "+=" operator.

Parameters to run the model:
- teams: number of teams
- solvers: ids of solver to test:
    - **CBC** is the PuLP default
    - **HiGHS** and **SCIP** are free
    - **CPLEX** is IBM's solver, requires license and could not be legally uploaded to docker container
- timeout

In [ ]:
from pulp import *
import argparse
from timeit import default_timer as timer
import json

# Command line parameters
parser = argparse.ArgumentParser(description="STS with MIP")
parser.add_argument("-n", "--teams", type=int, help="Number of teams")
parser.add_argument(
    "--solvers",
    nargs="+",
    type=str,
    default=["CBC"],
    choices=["CBC", "HiGHS", "gurobi", "CPLEX", "SCIP"],
    help="Solvers chosen to solve instance",
)
parser.add_argument(
    "-t", "--timeout", type=int, default=300, help="Solver timeout in seconds"
)

args = parser.parse_args()

n = args.teams  # number of teams
solvers = args.solvers  # List of solver ids
timeout = args.timeout  # Solver timeout

teams = range(n)  # Team identifiers
weeks = range(n - 1)  # Week identifiers
periods = range(n // 2)  # Period identifiers

**NOTE**: gurobi was not tested because the download and the license required to connect to the "university network".

Following code generates round-robin schedule: ($rb_{m, w}$ is week_matchups)

In [ ]:
# Round robin
week_matchups = []
for w in weeks:
    current_week_games = []
    current_week_games.append(tuple(sorted([n - 1, w])))

    for p in range(1, n // 2):
        team1 = (p + w) % (n - 1)
        team2 = (n - p + w - 1) % (n - 1)
        current_week_games.append(tuple(sorted([team1, team2])))
    week_matchups.append(current_week_games)

The following method takes as input the assigned decision variables and creates the output list of list schedule.

In [ ]:
def format_schedule(period_assign, home_away_assign, week_matchups, n):
    schedule = []
    for p in range(n // 2):
        period_games = []
        for w in range(n - 1):
            week_game = None
            for g in range(n // 2):
                if value(period_assign[w][p][g]) > 0.5:
                    i, j = week_matchups[w][g]
                    if value(home_away_assign[w][g]) > 0.5:
                        week_game = [j + 1, i + 1]
                    else:
                        week_game = [i + 1, j + 1]

            period_games.append(week_game)

        schedule.append(period_games)
    return schedule

For each solver provided by the user the model is run.  
The CPLEX solver is assumed to be in the the directory specified in the README.md file

In [ ]:
result = {}  # holds result for each solver

for solver_id in solvers:
    # Selecting solver
    match solver_id:
        case "CBC":
            solver = PULP_CBC_CMD(timeLimit=timeout)
        case "HiGHS":
            solver = HiGHS(timeLimit=timeout)
        case "gurobi":
            solver = GUROBI_CMD(timeLimit=timeout)
        case "CPLEX":
            solver = CPLEX_CMD(
                path="/opt/ibm/ILOG/CPLEX_Studio2211/cplex/bin/x86-64_linux/cplex",
                timeLimit=timeout,
            )
        case "SCIP":
            solver = SCIP_CMD(timeLimit=timeout)

Initialize the the MIP problem object:

In [ ]:
# Model
prob = LpProblem("STS problem", LpMinimize)

- period_assign corresponds to $match\_schedule_{w, p, m}$ from the report, equal to 1 iff match m is played in week w and period p
- home_away_assign corresponds to $flip_slot_{w, p}$ from the report, equal to 0 if order in $rb$ is preserved, 1 to reverse

In [ ]:
# Decision variables
period_assign = LpVariable.dicts(
    "PeriodAssignment", indices=(weeks, periods, periods), cat="Binary"
)  # period_assign[i, j, k] == 1 if in week i period j the game in position k in week_matchups is played out

home_away_assign = LpVariable.dicts(
    "HomeAwayAssignment", indices=(weeks, periods), cat="Binary"
)  # For (i, j) in week_matchups[week, game_index], value 0 for i vs j and value 1 for j vs i


Each period is assigned to one game and each game to one period constraints.  
NOTE: constraints are a linear combination (lpSum) and are added to the problem with "prob += ..." where prob is the problem object

In [ ]:
# Each period is assigned one game per week
for w in weeks:
    for p in periods:
        prob += lpSum([period_assign[w][p][g] for g in periods]) == 1

# Each game is assigned to one period
for w in weeks:
    for g in periods:
        prob += lpSum([period_assign[w][p][g] for p in periods]) == 1

- team_home_games corresponds to $H_t$, the home game counter.
- team_play_period corresponds to $TP_{t, w, p}$ from the report, its equal to 1 if team t plays in week w and period p.

In [ ]:
# Auxiliary variables
team_home_games = LpVariable.dicts(
    "HomeGamesCount", (teams), lowBound=0, upBound=n - 1, cat="Integer"
)
team_play_period = LpVariable.dicts(
    "TeamPlayerPeriod", indices=(teams, weeks, periods), cat="Binary"
)  # team_play_period[i,j,k] == 1 if team i in week j plays in period k


Linking together values of $match_schedule_{m, w, p}$ to those in $TP_{t, w, p}$ based on $rb$:

In [ ]:
# Link team_play_period to period_assign and week_matchups
for t in teams:
    for w in weeks:
        team_weekly_game = None  # Game team t plays in week w
        for game_idx in periods:
            team1, team2 = week_matchups[w][game_idx]
            if team1 == t or team2 == t:
                team_weekly_game = game_idx
                break

        for p in periods:
            # team_play_period[t][w][p] == 1 iff period-assign[w][p][game_idx] == 1
            prob += team_play_period[t][w][p] == period_assign[w][p][game_idx]


Constraint of each team playing at most twice in the same period:

In [ ]:
# Each team plays at most twice in the same period
for t in teams:
    for p_scheduled in periods:
        prob += lpSum([team_play_period[t][w][p_scheduled] for w in weeks]) <= 2

Objective variable initialization and addition to problem:

In [ ]:
# Objective variable
max_imbalance = LpVariable("MaxImbalance", lowBound=1, upBound=n - 1, cat="Integer")

prob += max_imbalance

Computing home games for each team and imposing max_imbalance greater than the absolute value of the difference (abs is decomposed into 2 inequalities)

The number of home games for a team is the number of times the team is assigned to the first slot and there is no slot flip plus the number of times the team is assigned to the second and there is a slot flip.

In [ ]:
# How much a team plays at home
for t in teams:
    prob += team_home_games[t] == lpSum(
        [
            (1 - home_away_assign[w][p])
            for w in weeks
            for p in periods
            if week_matchups[w][p][0] == t
        ]
    ) + lpSum(
        home_away_assign[w][p]
        for w in weeks
        for p in periods
        if week_matchups[w][p][1] == t
    )

for t in teams:
    # max_imbalance = max{|home_games - away_games|}
    prob += max_imbalance >= (2 * team_home_games[t] - (n - 1))
    prob += max_imbalance >= -(2 * team_home_games[t] - (n - 1))

timing the runtime of the solver, solve() runs the solver on the prob problem

In [ ]:
# Solve problem
start = timer()
prob.solve(solver)
end = timer()

Storing results at timeout.  
NOTE: the if statement in optimal is to counteract CPLEX not stopping at timeout (yeah, its a barbaric way to address the issue)

In [ ]:
# Solution output
match LpStatus[prob.status]:
    case "Optimal":  # Found optimal solution
        if (
            solver_id == "CPLEX" and int(end - start) > 300
        ):  # Fix for CPLEX sometimes not stopping at timeout
            result[solver_id] = {
                "time": timeout,
                "optimal": False,
                "obj": None,
                "sol": [],
            }
        else:
            result[solver_id] = {
                "time": int(end - start),
                "optimal": True,
                "obj": value(prob.objective),
                "sol": format_schedule(
                    period_assign, home_away_assign, week_matchups, n
                ),
            }
    case "Not optimal":  # No optimality guarantee
        result[solver_id] = {
            "time": timeout,
            "optimal": False,
            "obj": value(prob.objective),
            "sol": format_schedule(
                period_assign, home_away_assign, week_matchups, n
            ),
        }
    case "Not Solved":  # Timed out before finding a solution
        result[solver_id] = {
            "time": timeout,
            "optimal": False,
            "obj": None,
            "sol": [],
        }
    case "Infeasible":
        result[solver_id] = {
            "time": int(end - start),
            "optimal": False,
            "obj": None,
            "sol": None,
        }
    case "Unbounded":
        result[solver_id] = {
            "time": int(end - start),
            "optimal": False,
            "obj": "-inf",
            "sol": None,
        }

print(result)
# Storing result
with open("res/MIP/" + str(n) + ".json", "w") as file:
    json.dump(result, file)

## Final considerations
- Not using symmetry breaking constraints because the variable additions slow down the model instead of speeding up the search.
- SCIP is the best solver while CBC was the one with the worst performance.